<a href="https://colab.research.google.com/github/ArifQureshi3966/MLProject/blob/main/MFCC_with_GMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Feature Extraction

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp37-none-any.whl size=5887 sha256=a3565f11c71f04e21d26986b77a8369de5020662fe17b5d80883dbff458164a2
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [3]:
!pip install pickle-mixin

  Created wheel for pickle-mixin: filename=pickle_mixin-1.0.2-cp37-none-any.whl size=5999 sha256=35d055f83250abfe49c845de351ce7f85d8f01ead068957afc3d41e0ff31786f
  Stored in directory: /root/.cache/pip/wheels/cd/05/42/71de70fa36b9cbb7657bb5793a16f8028c1cdc1bdd3b8e1ac3
Successfully built pickle-mixin


In [ ]:
!pip install auto-sklearn

In [5]:
!pip install librosa

In [7]:
import numpy as np
from sklearn import preprocessing
import python_speech_features as mfcc
 
def calculate_delta(array):
    """Calculate and returns the delta of given feature vector matrix"""

    rows,cols = array.shape
    deltas = np.zeros((rows,20))
    N = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= N:
            if i-j < 0:
                first = 0
            else:
                first = i-j
            if i+j > rows -1:
                second = rows -1
            else:
                second = i+j
            index.append((second,first))
            j+=1
        deltas[i] = ( array[index[0][0]]-array[index[0][1]] + (2 * (array[index[1][0]]-array[index[1][1]])) ) / 10
    return deltas
 
def extract_features(audio,rate):
    """extract 20 dim mfcc features from an audio, performs CMS and combines
    delta to make it 40 dim feature vector"""   
 
    mfcc_feat = mfcc.mfcc(audio,rate, 0.025, 0.01,20,appendEnergy = True)
    mfcc_feat = preprocessing.scale(mfcc_feat)
    delta = calculate_delta(mfcc_feat)
    combined = np.hstack((mfcc_feat,delta))
    return combined

#Training Speaker Models

In [8]:
import pickle
import librosa
import numpy as np
from sklearn import mixture
import warnings
warnings.filterwarnings("ignore")
 
#path to training data
source   = "/content/drive/MyDrive/Selfdata/"  
 
#path where training speakers will be saved
dest = "/content/drive/MyDrive/gmm_models/"
train_file = "/content/drive/MyDrive/Selfdata/training_data.txt"
file_paths = open(train_file,'r')
 
count = 1
# Extracting features for each speaker (5 files per speakers)
features = np.asarray(())
for path in file_paths:
    path = path.strip()
    print (path)
 
    # read the audio
    audio,sr = librosa.load(source + path, sr=16000)
 
    # extract 40 dimensional MFCC & delta MFCC features
    vector   = extract_features(audio,sr)
 
    if features.size == 0:
        features = vector
    else:
        features = np.vstack((features, vector))
    # when features of 5 files of speaker are concatenated, then do model training
    if count == 5:
        gmm = mixture.GaussianMixture(n_components=16, covariance_type='diag',max_iter=200,n_init = 3)
        gmm.fit(features)
 
        # dumping the trained gaussian model
        picklefile = path.split("-")[0]+".gmm"
        pickle.dump(gmm,open(dest + picklefile,'wb'))
        print ('Modeling completed for speaker:',picklefile," with data point = ",features.shape)
        features = np.asarray(())
        count = 0
    count = count + 1

Ali-001/Ali (1).wav
Ali-001/Ali (2).wav
Ali-001/Ali (3).wav
Ali-001/Ali (4).wav
Ali-001/Ali (5).wav
Modeling completed for speaker: Ali.gmm  with data point =  (4820, 40)
Alsaba-002/Alsaba (1).wav
Alsaba-002/Alsaba (2).wav
Alsaba-002/Alsaba (3).wav
Alsaba-002/Alsaba (4).wav
Alsaba-002/Alsaba (5).wav
Modeling completed for speaker: Alsaba.gmm  with data point =  (4354, 40)
Amna-003/Amna (1).wav
Amna-003/Amna (2).wav
Amna-003/Amna (3).wav
Amna-003/Amna (4).wav
Amna-003/Amna (5).wav
Modeling completed for speaker: Amna.gmm  with data point =  (4197, 40)
Aneela-004/Aneela (1).wav
Aneela-004/Aneela (2).wav
Aneela-004/Aneela (3).wav
Aneela-004/Aneela (4).wav
Aneela-004/Aneela (5).wav
Modeling completed for speaker: Aneela.gmm  with data point =  (4992, 40)
Arif-005/Arif (1).wav
Arif-005/Arif (2).wav
Arif-005/Arif (3).wav
Arif-005/Arif (4).wav
Arif-005/Arif (5).wav
Modeling completed for speaker: Arif.gmm  with data point =  (5271, 40)
Bareera-006/bareera (1).wav
Bareera-006/bareera (2).wav
B

#Evaluating Performance on Test set.

In [9]:
import os
import librosa
import pickle
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import time
 
#path to training data
source   = "/content/drive/MyDrive/Selfdata/"
modelpath = "/content/drive/MyDrive/gmm_models/"
test_file = "/content/drive/MyDrive/Selfdata/testing_data.txt"
file_paths = open(test_file,'r')
 
gmm_files = [os.path.join(modelpath,fname) for fname in
              os.listdir(modelpath) if fname.endswith('.gmm')]
 
#Load the Gaussian gender Models
models    = [pickle.load(open(fname,'rb')) for fname in gmm_files]
speakers   = [fname.split("/")[-1].split(".gmm")[0] for fname
              in gmm_files]
error = 0
total_sample = 0.0
# Read the test directory and get the list of test audio files
for path in file_paths:   
    total_sample += 1.0
    path = path.strip()
    print (path)
    audio,sr = librosa.load(source + path, sr=16000)
    vector   = extract_features(audio,sr)
 
    log_likelihood = np.zeros(len(models)) 
 
    for i in range(len(models)):
        gmm    = models[i]  #checking with each model one by one
        scores = np.array(gmm.score(vector))
        log_likelihood[i] = scores.sum()
 
    winner = np.argmax(log_likelihood)
    print ("\tdetected as - ", speakers[winner])

    checker_name = path.split("-")[0]
    #checker_name = path.split("/")[-1].split(" ")[0]
    if speakers[winner]!= checker_name:
      error += 1
    time.sleep(1.0)

Ali-001/Ali (6).wav
	detected as -  Ali
Ali-001/Ali (7).wav
	detected as -  Ali
Ali-001/Ali (8).wav
	detected as -  Ali
Ali-001/Ali (9).wav
	detected as -  Ali
Ali-001/Ali (10).wav
	detected as -  Ali
Alsaba-002/Alsaba (6).wav
	detected as -  Alsaba
Alsaba-002/Alsaba (7).wav
	detected as -  Alsaba
Alsaba-002/Alsaba (8).wav
	detected as -  Alsaba
Alsaba-002/Alsaba (9).wav
	detected as -  Alsaba
Alsaba-002/Alsaba (10).wav
	detected as -  Alsaba
Amna-003/Amna (6).wav
	detected as -  Amna
Amna-003/Amna (7).wav
	detected as -  Amna
Amna-003/Amna (8).wav
	detected as -  Amna
Amna-003/Amna (9).wav
	detected as -  Amna
Amna-003/Amna (10).wav
	detected as -  Amna
Aneela-004/Aneela (6).wav
	detected as -  Aneela
Aneela-004/Aneela (7).wav
	detected as -  Aneela
Aneela-004/Aneela (8).wav
	detected as -  Aneela
Aneela-004/Aneela (9).wav
	detected as -  Aneela
Aneela-004/Aneela (10).wav
	detected as -  Aneela
Arif-005/Arif (6).wav
	detected as -  Arif
Arif-005/Arif (7).wav
	detected as -  Arif
Arif-

In [10]:
print ('Error =',error,'Total Test Samples =', total_sample)
accuracy = ((total_sample - error) / total_sample) * 100
print("The Accuracy Percentage for the current testing Performance with MFCC with GMM is : ", accuracy, "%")

Error = 0 Total Test Samples = 65.0
The Accuracy Percentage for the current testing Performance with MFCC with GMM is :  100.0 %
